---

# Question 1

Solving for flow rate


In [85]:
import numpy as np

# Define the coefficient matrix A and the constants vector b
A = np.array([
    [-1,    .1,     0,      0],
    [1,     -1,     0,      0.2],
    [0,     .2,     -1,     0],
    [0,     .7,     0,      -1]
])

# Right-hand side constants vector (all zero in this case)
b = np.array([-1, 0, 0, 0])

# Solve the system using np.linalg.solve if it’s square and has a unique solution
# If it does not have a unique solution, we can use np.linalg.lstsq to get a least-squares solution
try:
    lam = np.linalg.solve(A, b)
except np.linalg.LinAlgError:
    # If matrix A is singular, use least-squares solution
    lam, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

# The flow rates are the following 
lam


array([1.13157895, 1.31578947, 0.26315789, 0.92105263])


The solution for the outflow values is:

\begin{align*}
\lambda_1 &= 1.13157895 \\
\lambda_2 &= 1.31578947 \\
\lambda_3 &= 0.26315789 \\
\lambda_4 &= 0.92105263 \\
\end{align*}

---

---

Now let's get the utilization

In [86]:
# Define the vector te
te = np.array([0.8, 0.7, 3, 1])

# Calculate the vector u as the product of lam and te
u = lam * te
u

array([0.90526316, 0.92105263, 0.78947368, 0.92105263])

\begin{align*}
u_1 &= 0.90526316 \\
u_2 &= 0.92105263 \\
u_3 &= 0.78947368 \\
u_4 &= 0.92105263 \\
\end{align*}


---

Let's now find the SCV for arrivals

In [87]:
from sympy import symbols, Eq, solve

# Define symbols for the variables
scv_a_1, scv_a_2, scv_a_3, scv_a_4 = symbols('scv_a_1, scv_a_2, scv_a_3, scv_a_4')

$$C_a^2(1)$$

In [88]:
p21=0.1
scv_e_2=0.75

eq1 = Eq(scv_a_1,1/lam[0]+(lam[1]*p21/lam[0])*(p21*((1-(u[1])**2)*scv_a_2+(u[1])**2*scv_e_2)+1-p21))

$$ C_a^2(2)$$

In [89]:
p42=0.2
scv_e_4=0.5

p12=1
scv_e_1=1.5

eq2 = Eq(scv_a_2,
         lam[3]*p42/lam[1]*(p42*((1-(u[3])**2)*scv_a_4+(u[3])**2*scv_e_4)+1-p42)+
            lam[0]*p12/lam[1]*(p12*((1-(u[0])**2)*scv_a_1+(u[0])**2*scv_e_1)+1-p12))

$$ C_a^2(3)$$

In [90]:
p23 = 0.2

eq3 = Eq(scv_a_3, 
         lam[1]*p23/lam[2]*(p23*((1-(u[1])**2)*scv_a_2+(u[1])**2*scv_e_2)+1-p23))

$$ C_a^2(4)$$

In [91]:
p24 = 0.7
scv_e_2 = 0.75
eq4 = Eq(scv_a_4, 
         lam[1]*p24/lam[3]*(p24*((1-(u[1])**2)*scv_a_2+(u[1])**2*scv_e_2)+1-p24))

In [92]:
# Solve the system of equations
solution = solve((eq1, eq2, eq3, eq4), (scv_a_1, scv_a_2, scv_a_3, scv_a_4))

print("Solution:", solution)
scv_a_1 = solution[scv_a_1]
scv_a_2 = solution[scv_a_2]
scv_a_3 = solution[scv_a_3]
scv_a_4 = solution[scv_a_4]

Solution: {scv_a_1: 0.998133040094418, scv_a_2: 1.33974178577721, scv_a_3: 0.967888289623990, scv_a_4: 0.887609013683965}


\begin{align*}
C_a^2(1) &= 0.998133040094418 \\
C_a^2(2) &= 1.33974178577721 \\
C_a^2(3) &= 0.967888289623990 \\
C_a^2(4) &= 0.887609013683965 \\
\end{align*}


---

Let's now work on the WIP of each station

Now let's solve for Cycle Time

$$CT_s(k) = \frac{C_a^2(k) + C_e^2(k)}{2} \cdot \frac{u_k}{1 - u_k} \cdot t_e(k) + t_e(k)$$


In [93]:
CT_1=(scv_a_1+scv_e_1)/2 * u[0]/(1-u[0]) * te[0]+te[0]
CT_1

10.3484196199164

In [94]:
CT_2=(scv_a_2+scv_e_2)/2 * u[1]/(1-u[1]) * te[1]+te[1]
CT_2

9.23311229192359

In [95]:
CT_3=(scv_a_3+scv_e_2)/2 * u[2]/(1-u[2]) * te[2]+te[2]
CT_3

12.6631216291349

In [96]:
CT_4=(scv_a_4+scv_e_4)/2 * u[3]/(1-u[3]) * te[3]+te[3]
CT_4

9.09438591315646

In [97]:
print("CT_1:", CT_1, "CT_2:", CT_2, "CT_3:", CT_3, "CT_4:", CT_4)

CT_1: 10.3484196199164 CT_2: 9.23311229192359 CT_3: 12.6631216291349 CT_4: 9.09438591315646


---

Now let's get the WIP of each workstation

$$\text{WIP}_s(k) = \lambda_k \cdot CT_s(k)$$


In [98]:
WIP_1=CT_1*lam[0]
WIP_1

11.7100537804318

In [99]:
WIP_2=CT_2*lam[1]
WIP_2

12.1488319630573

In [100]:
WIP_3=CT_3*lam[2]
WIP_3

3.33240042871972

In [101]:
WIP_4=CT_4*lam[3]
WIP_4

8.37640807790726

In [102]:
print("WIP_1:", WIP_1, "WIP_2:", WIP_2, "WIP_3:", WIP_3, "WIP_4:", WIP_4)

WIP_1: 11.7100537804318 WIP_2: 12.1488319630573 WIP_3: 3.33240042871972 WIP_4: 8.37640807790726


In [103]:
WIP = WIP_1 + WIP_2 + WIP_3 + WIP_4
WIP

35.5676942501161

---

Cycle Time of a Job in the System

$$CT_s=\frac{WIP_s}{TH_s}$$

In [105]:
TH=1
CT=WIP/1
CT

35.5676942501161

---
Profit of the System

In [109]:
lam_3_exit=lam[2]
lam_3_exit

0.26315789473684215

In [110]:
lam_4_exit=lam[3]*.8
lam_4_exit

0.736842105263158

In [111]:
lam_exit=lam_3_exit+lam_4_exit
lam_exit

1.0